In [8]:
# import pandas, shutil, os
import pandas as pd
import shutil
import os
import random
import math
import glob

In [9]:
# allow multiple output per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print("Multiple outputs per cell")

Multiple outputs per cell


In [10]:
# variables to determine the source and destination paths
TRAIN_DIRECTORY = 'C:/Users/akash/Desktop/Akash/MSOEML/NIH_data/images' # no need to put a forward slash at the front of directory name

TEST_DIRECTORY = 'C:/Users/akash/Desktop/Akash/MSOEML/NIH_data/imagesTest'

In [11]:
# read in data and inspect it
images = pd.read_csv('NIH.csv')

In [12]:
# make the Image Index column as the index
images.set_index('Image Index', inplace=True)
images

,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
Image Index,,,,,,,,,,
00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168
...,...,...,...,...,...,...,...,...,...,...
00030801_001.png,Mass|Pneumonia,1,30801,38,M,PA,2048,2500,0.168,0.168
00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,0.168
00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168


## Patients with Multiple Diseases

In [1]:
for index, row in images[images['Finding Labels'].str.contains('\|')].iterrows():
    # get the image name
    image_name = row.index

    # get the image label
    image_label = row['Finding Labels']

    # split the image label into multiple labels
    image_labels = image_label.split('|')

    print(image_labels)

    # create a new folder for each label
    for label in image_labels:
        # create a new folder for each label
        new_folder_name = label
        new_folder_path = os.path.join(TRAIN_DIRECTORY, new_folder_name)
        if not os.path.exists(new_folder_path):
            os.makedirs(new_folder_path)
        image_path = os.path.join(TRAIN_DIRECTORY, image_name)
        
        # copy the image to the new folder
        shutil.move(image_path, new_folder_path)

NameError: name 'images' is not defined

## Drop the rows that you just sorted, now deal with patients with only one disease each

In [6]:
# filter out multiple conditions & print the remaining ones
images.drop(images[images['Finding Labels'].str.contains('\|')].index, inplace=True)
images

diseases = images['Finding Labels'].unique()
diseases

,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
Image Index,,,,,,,,,,
00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168
00000003_002.png,Hernia,1,3,75,F,PA,2048,2500,0.168,0.168
00000003_004.png,Hernia,3,3,77,F,PA,2500,2048,0.168,0.168
...,...,...,...,...,...,...,...,...,...,...
00030801_000.png,No Finding,0,30801,38,M,PA,2500,2048,0.168,0.168
00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,0.168
00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168


array(['Cardiomegaly', 'No Finding', 'Hernia', 'Infiltration', 'Nodule',
       'Emphysema', 'Effusion', 'Atelectasis', 'Mass', 'Pneumothorax',
       'Pleural_Thickening', 'Fibrosis', 'Consolidation', 'Edema',
       'Pneumonia'], dtype=object)

In [11]:
for disease in diseases:
    if not os.path.exists(os.path.join(TRAIN_DIRECTORY, disease)):
        os.makedirs(os.path.join(TRAIN_DIRECTORY, disease))

for disease in diseases:
    if not os.path.exists(os.path.join(TEST_DIRECTORY, disease)):
        os.makedirs(os.path.join(TEST_DIRECTORY, disease))

print("Confirming that the directories exist")

print("Training Directory")
print(glob.glob(f'{TRAIN_DIRECTORY}/*/'))
print("Validation Directory")
print(glob.glob(f'{TEST_DIRECTORY}/*/'))

Confirming that the directories exist
Training Directory
['/Volumes/90OL67YGN/images\\Atelectasis\\', '/Volumes/90OL67YGN/images\\Cardiomegaly\\', '/Volumes/90OL67YGN/images\\Consolidation\\', '/Volumes/90OL67YGN/images\\Edema\\', '/Volumes/90OL67YGN/images\\Effusion\\', '/Volumes/90OL67YGN/images\\Emphysema\\', '/Volumes/90OL67YGN/images\\Fibrosis\\', '/Volumes/90OL67YGN/images\\Hernia\\', '/Volumes/90OL67YGN/images\\Infiltration\\', '/Volumes/90OL67YGN/images\\Mass\\', '/Volumes/90OL67YGN/images\\No Finding\\', '/Volumes/90OL67YGN/images\\Nodule\\', '/Volumes/90OL67YGN/images\\Pleural_Thickening\\', '/Volumes/90OL67YGN/images\\Pneumonia\\', '/Volumes/90OL67YGN/images\\Pneumothorax\\']
Validation Directory
['/Volumes/90OL67YGN/imagesTest\\Atelectasis\\', '/Volumes/90OL67YGN/imagesTest\\Cardiomegaly\\', '/Volumes/90OL67YGN/imagesTest\\Consolidation\\', '/Volumes/90OL67YGN/imagesTest\\Edema\\', '/Volumes/90OL67YGN/imagesTest\\Effusion\\', '/Volumes/90OL67YGN/imagesTest\\Emphysema\\', '/

In [9]:
for fileName in os.listdir(TRAIN_DIRECTORY):
    if fileName in diseases:
        continue
    if fileName not in images.index:
        continue
    
    # move the file to the appropriate directory
    sp = os.path.join(TRAIN_DIRECTORY, fileName)
    disease = images.loc[fileName]['Finding Labels']
    dp = os.path.join(TRAIN_DIRECTORY, disease)
    _ = shutil.move(sp, dp)